# 🎨 Neural Style Transfer - Interactive Demo

This notebook demonstrates neural style transfer using a feedforward Transformer network.

You can:
- Upload your own content image, or choose one from examples.
- Select a style from pretrained models (e.g., Starry Night, Mosaic).
- Stylize the image and view the result interactively.
- Compare before and after side by side.
- Save the stylized image.

In [ ]:
import os
import torch
import ipywidgets as widgets
import matplotlib.pyplot as plt
from datetime import datetime
from pathlib import Path
from IPython.display import display
from utils.utils import prepare_img, post_process_image
from utils.jupyter_parsing import parse_uploaded_file
from models.definitions.transformer_net import TransformerNet

## 🖼️ User Inputs: Select or Upload Content Image, and Choose Style

In [ ]:
# Directories
content_path = "data/input"
output_path = "data/output"
model_path = "models/binaries"

# Available files
available_images = sorted([f for f in os.listdir(content_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
available_styles = sorted([f for f in os.listdir(model_path) if f.endswith('.pth')])

# Upload or choose image
use_uploaded = widgets.Checkbox(value=False, description="Upload your own image")
uploader = widgets.FileUpload(accept=".jpg,.png", multiple=False)

# Dropdowns
image_dropdown = widgets.Dropdown(options=available_images, description="Choose Image:")
style_dropdown = widgets.Dropdown(options=available_styles, description="Choose Style:")

# Display widgets
display(use_uploaded, uploader)
display(image_dropdown, style_dropdown)

### 🗂️ Resolve Inputs

In [ ]:
if use_uploaded.value and uploader.value:
    content_img_name = parse_uploaded_file(uploader, content_path)
    print(f"Uploaded image saved as: {content_img_name}")
else:
    content_img_name = image_dropdown.value

style_model_name = style_dropdown.value

## 🧠 Stylization Logic

In [ ]:
# Set device
if torch.backends.mps.is_available():
    device = torch.device("mps")    # for Apple Silicone
elif torch.cuda.is_available():
    device = torch.device("cuda")   # for Nvidia
else:
    device = torch.device("cpu")

# Load model
model = TransformerNet().to(device)
checkpoint = torch.load(os.path.join(model_path, style_model_name), map_location=device)
model.load_state_dict(checkpoint['state_dict'])
model.eval()

# Prepare image
img_path = os.path.join(content_path, content_img_name)
input_tensor = prepare_img(img_path, target_shape=500, device=device)

# Stylize
with torch.no_grad():
    output_tensor = model(input_tensor).cpu().numpy()[0]

# Convert to displayable format
stylized_img = post_process_image(output_tensor)

## 🔍 View: Before vs After

In [ ]:
import cv2
original_img = cv2.imread(img_path)[:, :, ::-1]  # BGR to RGB

# Display side by side
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(original_img)
axes[0].set_title("Original Image")
axes[0].axis("off")
axes[1].imshow(stylized_img)
axes[1].set_title("Stylized Image")
axes[1].axis("off")
plt.tight_layout()
plt.show()

## 💾 Save Stylized Image

In [ ]:
save_name = f"stylized_{Path(content_img_name).stem}_{Path(style_model_name).stem}.jpg"
save_path = os.path.join(output_path, save_name)

os.makedirs(output_path, exist_ok=True)
cv2.imwrite(save_path, cv2.cvtColor(stylized_img, cv2.COLOR_RGB2BGR))
print(f"Stylized image saved to: {save_path}")